Generate a gemini api key and put in secrets of colab {"gemini_api" : "xxxxxxxxxxxxxxxx"}<br>
RUN ALL AND GET RESPONSE

In [ ]:
!pip -q install tqdm
!pip -q install pandas
!pip -q install sentence-transformers==2.2.2
!pip -q install llama-index==0.9.29
!pip -q install loguru==0.7.0
!pip -q install rich
!pip -q install tiktoken
!pip -q install chromadb

In [16]:
%load_ext autoreload
%autoreload 2

#standard libraries
import sys
sys.path.append('../')

import json
import os
import time
from typing import List, Tuple
from math import ceil

#external libraries
import pandas as pd
import numpy as np
from rich import print
from rich.pretty import pprint #nifty library for pretty printing
from torch import cuda
from tqdm import tqdm

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/paramjeetn/Doctor_Copilot_V1.git

In [29]:
import json
import chromadb
#change path accordingly for local and colab
path = '/content/Doctor_Copilot_V1/ChromaDb'
client = chromadb.PersistentClient(path)
client.heartbeat()



1719773236786413833

In [30]:
collection = client.get_collection("Doctor_Copilot_chromaDB_Collection")


In [31]:

collection.query(
    query_texts=["Prevention of diabetes in older adults"],
    n_results=2,
)


{'ids': [['d_11_Summary', 'd_2_cont_2']],
 'distances': [[0.7675116062164307, 0.8127467036247253]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['The approach to managing diabetes in older adults is differentiated by their health status, ranging from healthy individuals to those at the end of life, with tailored A1C goals to match each scenario. For healthy older adults, the target A1C is below 7-7.5%, while more complex cases with multiple chronic illnesses or cognitive impairments aim for less stringent targets like <8.0%. Treatment in skilled nursing facilities focuses on maintaining glucose levels between 100–200 mg/dL, avoiding strict A1C adherence. In very complex or poor health situations, the focus shifts from A1C levels to preventing hypoglycemia and managing symptomatic hyperglycemia, emphasizing comfort. Medication strategies are adapted to the individual’s ability to manage diabetes, with options to simplify or intensify regimens based on changes in he

In [32]:
def get_related_doc(query):
    result = collection.query(
        query_texts=[query],
        n_results=3,
    )
    return result
print(get_related_doc("Prevention of diabetes in older adults"))

{
    'ids': [['d_11_Summary', 'd_2_cont_2', 'd_8_Summary']],
    'distances': [[0.7675116062164307, 0.8127467036247253, 0.8671669960021973]],
    'metadatas': [[None, None, None]],
    'embeddings': None,
    'documents': [
        [
            'The approach to managing diabetes in older adults is differentiated by their health status, ranging 
from healthy individuals to those at the end of life, with tailored A1C goals to match each scenario. For healthy 
older adults, the target A1C is below 7-7.5%, while more complex cases with multiple chronic illnesses or cognitive
impairments aim for less stringent targets like <8.0%. Treatment in skilled nursing facilities focuses on 
maintaining glucose levels between 100–200 mg/dL, avoiding strict A1C adherence. In very complex or poor health 
situations, the focus shifts from A1C levels to preventing hypoglycemia and managing symptomatic hyperglycemia, 
emphasizing comfort. Medication strategies are adapted to the individual’s ability to manage diabetes, with options
to simplify or intensify regimens based on changes in health status, hypoglycemia risk, and overall treatment 
goals. Continuous glucose monitoring and other technologies are considered to reduce hypoglycemia, alongside 
medications that address cardiovascular risks irrespective of glycemic control. This patient-centered approach 
prioritizes flexible treatment goals to enhance quality of life and reduce treatment burdens in older adults.',
            '• Symptoms suggestive of diabetes or changes in risk (B) Diet and Physical Activity Recommendations 
for Adults at Risk for Type 2 Diabetes (i) Follow a Healthy Eating Pattern • Emphasize whole grains, legumes, nuts,
fruits, and vegetables and minimize refined and processed foods • A variety of healthy eating patterns include: » 
Mediterranean-style » Low-carbohydrate » Vegetarian or plant-based » DASH (Dietary Approaches to Stop Hypertension)
(ii) Get Regular Physical Activity • ≥150 min/week of moderate-intensity physical activity, such as brisk walking •
May include resistance or strength training • Break up prolonged sedentary time (iii) Where to Refer: • Refer 
adults with overweight or obesity who are at high risk for type 2 diabetes to a recognized diabetes prevention 
lifestyle change program (cdc.gov/diabetes/prevention/find-a-program.html). (iv) What is the Diabetes Prevention 
Program? • The Diabetes Prevention Program (DPP) study demonstrated that intensive lifestyle intervention could 
reduce the risk of type diabetes by 58% over 3 years.',
            "The comprehensive medical evaluation for diabetes management involves a holistic approach that 
prioritizes collaborative goal setting tailored to individual preferences and values. An interprofessional team 
including specialists from various medical and community fields supports ongoing care. Key components include 
regular screening for common and serious comorbidities, such as autoimmune disorders and chronic kidney disease, 
particularly focusing on diabetes-related complications like nephropathy. For instance, people with type 1 diabetes
are screened for autoimmune thyroid disease and other conditions soon after diagnosis, while both type 1 and type 2
diabetes patients undergo regular assessments for bone health, cardiovascular risks, and liver disease. Effective 
management incorporates lifestyle interventions, pharmacotherapy aligned with specific health risks, and regular 
monitoring of treatment efficacy and safety. The approach also includes a detailed review of the patient's medical,
family, and social history, physical examinations, laboratory evaluations, and vaccinations, ensuring a thorough 
assessment and optimal management of diabetes and its associated conditions."
        ]
    ],
    'uris': None,
    'data': None,
    'included': ['metadatas', 'documents', 'distances']
}

In [33]:
!pip install -q -U google-generativeai

In [9]:
# Import the Python SDK
import google.generativeai as genai
from google.colab import userdata
gemini_api_key = userdata.get('gemini_api')
# Used to securely store your API key
from google.colab import userdata

genai.configure(api_key=gemini_api_key)

In [34]:
model = genai.GenerativeModel('gemini-pro')

In [35]:
query_text = "What are the specific recommendations for screening and treatment of nonalcoholic fatty liver disease (NAFLD) in people with type 2 diabetes who are over 50 years old?"
query_related_doc=get_related_doc(query_text)
augmented_prompt = query_text + "\n Related Documents: " + str(query_related_doc['documents'])

response = model.generate_content(query_text)
print(response.text)

**Screening**

* All people with type 2 diabetes over 50 years of age should be screened for NAFLD.
* Screening can be done using:
    * Non-invasive tests: Fibroscan, FibroMeter, liver elastography
    * Liver function tests (LFTs) and metabolic panel
    * Imaging tests: Ultrasound, MRI, CT scan

**Treatment**

* **Lifestyle modifications:**
    * Weight loss of at least 5-10%
    * Regular exercise
    * Healthy diet (Mediterranean or DASH diet)
    * Smoking cessation
    * Alcohol intake moderation

* **Medications:**
    * Pioglitazone (a thiazolidinedione) may be considered if blood sugar control is stable.
    * Metformin is not recommended for NAFLD treatment.

* **Monitoring:**
    * Monitor liver function tests and fibrosis every 6-12 months to assess response to treatment.
    * Consider liver biopsy if non-invasive tests are inconclusive or suggest advanced fibrosis.

**Specific Considerations for People over 50 Years Old**

* Age is a risk factor for more advanced NAFLD and liver fibrosis.
* Comorbidities such as cardiovascular disease, kidney disease, and obesity are common in this population and may 
influence treatment decisions.
* Liver fibrosis progresses more slowly in older adults compared to younger adults.

**Additional Recommendations**

* Avoid hepatotoxic medications.
* Vaccinate against hepatitis A and B.
* Refer patients with suspected advanced fibrosis or cirrhosis to a liver specialist.
* Encourage participation in clinical trials to explore new treatment options.